<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/010_Mask2Former_ObtenerDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
### Importación de librerías estándar y especializadas para procesamiento de imágenes, manejo de datos y visualización ###

### Librería principal para computación con tensores y modelos de deep learning (PyTorch)
import torch
### Operaciones matemáticas y manejo de arrays multidimensionales
import numpy as np
### OpenCV: procesamiento de imágenes y video
import cv2
### Manejo de datos en formato JSON
import json
### Funciones relacionadas con el tiempo (ej. timestamps, delays)
import time
### Interacción con el sistema operativo (rutas, archivos, etc.)
import os
### Manejo de fechas y horas con precisión
from datetime import datetime
### Manejo de rutas de archivos de forma más robusta que con strings
from pathlib import Path
### Generación de hashes (útil para verificar integridad o crear IDs únicos)
import hashlib
### Decorador para crear clases de datos de forma concisa
from dataclasses import dataclass
### Tipado estático para mejorar legibilidad y mantenimiento
from typing import Dict, List, Any, Optional

### Importación de modelos y procesadores de imágenes desde Hugging Face Transformers ###
from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation
### AutoImageProcessor: preprocesamiento automático de imágenes para modelos específicos
### AutoModelForUniversalSegmentation: modelo para segmentación semántica de imágenes

### Librerías adicionales para manejo de imágenes y visualización ###

### PIL: manipulación de imágenes (abrir, convertir, analizar estadísticas)
from PIL import Image, ImageStat
### Visualización de datos e imágenes
import matplotlib.pyplot as plt
### Barra de progreso para loops largos (mejora experiencia visual en ejecución)
from tqdm import tqdm

### Montaje de Google Drive en entorno Colab para acceder a archivos almacenados en la nube ###
from google.colab import drive
### Solicita autorización para montar Google Drive en la ruta especificada
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
### Clase de configuración centralizada para definir rutas, parámetros y modelos utilizados en el proyecto ###
@dataclass
class Config:

    ### Ruta al conjunto de datos de entrada (imágenes a procesar) ###
    DATASET_PATH: Path = Path("/content/drive/MyDrive/TFM/mask2former/imagenes")

    ### Ruta donde se guardarán los resultados del procesamiento (segmentaciones, métricas, etc.) ###
    OUTPUT_PATH: Path = Path("/content/drive/MyDrive/TFM/mask2former/resultados")

    ### Diccionario de umbrales predefinidos para filtrar o evaluar resultados de segmentación.
    ### Cada clave representa un perfil de sensibilidad:
    ### - 'ultra': extremadamente sensible (detecta incluso mínimos cambios)
    ### - 'agresivo': sensibilidad alta
    ### - 'normal': sensibilidad media
    ### - 'conservador': sensibilidad baja (solo cambios significativos)
    UMBRALES = {
        'ultra': [0.0001, 0.001, 0.01, 0.1],
        'agresivo': [0.001, 0.01, 0.05, 0.1, 0.3],
        'normal': [0.01, 0.1, 0.3, 0.5],
        'conservador': [0.3, 0.5, 0.7]
    }

    ### Lista de modelos de segmentación disponibles.
    ### Cada modelo tiene diferentes arquitecturas y está entrenado en distintos datasets:
    ### - COCO Instance: segmentación de objetos individuales
    ### - ADE Semantic: segmentación semántica (clases por píxel)
    MODELOS = [
        "facebook/mask2former-swin-large-coco-instance",
        "facebook/mask2former-swin-base-ade-semantic",
        "facebook/mask2former-swin-small-coco-instance"
    ]


In [3]:
### Se crea una instancia de la clase Config para acceder a rutas, modelos y parámetros definidos ###
config = Config()

### Se asegura que la carpeta de salida exista; si no existe, se crea automáticamente.
### El parámetro `exist_ok=True` evita errores si la carpeta ya está creada ###
os.makedirs(config.OUTPUT_PATH, exist_ok=True)

Setup completado


In [4]:
### Clase que agrupa funciones utilitarias reutilizables para el procesamiento de imágenes y manejo de archivos ###
class Utils:

    ### Carga todas las imágenes desde una ruta dada, buscando recursivamente archivos con extensiones válidas.
    ### Parámetros:
    ### - ruta (str): ruta base donde buscar imágenes
    ### Retorna:
    ### - Lista de rutas (str) de imágenes encontradas
    @staticmethod
    def cargar_imagenes(ruta: str) -> List[str]:
        path = Path(ruta)
        extensiones = (".jpg", ".png", ".jpeg")
        imagenes = [str(p) for p in path.glob("**/*") if p.suffix.lower() in extensiones]
        print(f"Encontradas {len(imagenes)} imágenes")
        return imagenes

    ### Abre una imagen desde disco, la convierte a RGB y la redimensiona si excede el tamaño máximo.
    ### Parámetros:
    ### - ruta (str): ruta del archivo de imagen
    ### - max_size (int): tamaño máximo permitido para ancho/alto (por defecto 1024)
    ### Retorna:
    ### - Imagen PIL preparada para procesamiento
    @staticmethod
    def preparar_imagen(ruta: str, max_size: int = 1024) -> Image.Image:
        img = Image.open(ruta).convert("RGB")
        if max(img.size) > max_size:
            img.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
        return img

    ### Calcula un hash MD5 de los contenidos binarios del archivo.
    ### Útil para identificar imágenes de forma única o evitar duplicados.
    ### Parámetros:
    ### - ruta (str): ruta del archivo
    ### Retorna:
    ### - Cadena con los primeros 8 caracteres del hash MD5
    @staticmethod
    def calcular_hash(ruta: str) -> str:
        return hashlib.md5(open(ruta, 'rb').read()).hexdigest()[:8]

    ### Guarda un objeto Python (lista, diccionario, etc.) en un archivo JSON.
    ### Parámetros:
    ### - datos (Any): objeto a guardar
    ### - archivo (str): ruta del archivo destino
    ### No retorna nada, pero imprime confirmación en consola.
    @staticmethod
    def guardar_json(datos: Any, archivo: str) -> None:
        with open(archivo, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False)
        print(f"Guardado: {archivo}")


Utilidades definidas


In [5]:
### Clase DetectorPersonas
### Esta clase encapsula la lógica para detectar personas en imágenes utilizando modelos de segmentación universal
### de Hugging Face. Es compatible con modelos de segmentación semántica (por píxel) e instancia (por objeto).
###
### Funcionalidades principales:
### - Carga automática del modelo y procesador desde Hugging Face
### - Identificación dinámica de la clase "persona"
### - Inferencia sobre imágenes PIL
### - Post-procesamiento adaptativo según el tipo de modelo
### - Evaluación por múltiples umbrales de sensibilidad
### - Liberación de memoria para entornos con GPU limitada
class DetectorPersonas:
    ### Constructor de la clase
    ### Parámetros:
    ### - modelo_name (str): nombre del modelo Hugging Face a utilizar. Debe ser compatible con segmentación universal.
    def __init__(self, modelo_name: str):
        ### Guarda el nombre del modelo para referencia interna
        self.modelo_name = modelo_name

        ### Selección automática del dispositivo: GPU si está disponible, de lo contrario CPU
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        print(f"###### Cargando: {modelo_name}")

        ### Carga el procesador de imágenes asociado al modelo desde Hugging Face
        ### Este objeto se encarga de convertir imágenes PIL en tensores listos para el modelo
        self.processor = AutoImageProcessor.from_pretrained(modelo_name)

        ### Carga el modelo de segmentación universal desde Hugging Face
        ### Puede ser de tipo semántico (por píxel) o de instancia (por objeto)
        self.model = AutoModelForUniversalSegmentation.from_pretrained(modelo_name)

        ### Mueve el modelo al dispositivo seleccionado y lo pone en modo evaluación
        self.model.to(self.device)
        self.model.eval()

        ### Determina si el modelo es de segmentación semántica
        ### Se basa en la presencia de palabras clave como 'ade' o 'semantic' en el nombre del modelo
        self.es_semantico = ('ade' in modelo_name.lower() or 'semantic' in modelo_name.lower())

        ### Si el modelo tiene un diccionario de clases (id2label), se intenta identificar la clase "persona"
        if hasattr(self.model.config, 'id2label'):
            self.id2label = self.model.config.id2label
            print(f"Clase 0: {self.id2label.get(0, 'DESCONOCIDA')}")

            ### Búsqueda de la clase "persona" en el diccionario de etiquetas
            ### Se asigna el ID correspondiente a la clase que contenga 'person' o 'people'
            self.clase_persona = 0
            for clase_id, nombre in self.id2label.items():
                if 'person' in nombre.lower() or 'people' in nombre.lower():
                    self.clase_persona = clase_id
                    print(f"Clase persona encontrada: {clase_id} = {nombre}")
                    break
        else:
            ### Si no hay diccionario de clases, se asume por defecto que la clase "persona" es la 0
            self.clase_persona = 0

        print(f"Modelo cargado en {self.device} (semántico: {self.es_semantico})")

    ### Método detectar_en_imagen
    ### Realiza inferencia sobre una imagen PIL y aplica post-procesamiento según el tipo de modelo
    ###
    ### Parámetros:
    ### - imagen (PIL.Image): imagen a procesar. Debe estar en formato RGB y con tamaño adecuado.
    ### - umbrales (List[float]): lista de umbrales de sensibilidad. Cada umbral representa el mínimo requerido
    ###   para considerar que hay presencia significativa de personas. En modelos semánticos se interpreta como
    ###   porcentaje de píxeles, en modelos de instancia como score mínimo de confianza.
    ###
    ### Retorna:
    ### - Dict con resultados por umbral. Cada entrada incluye:
    ###   - personas: número de personas detectadas
    ###   - total: número total de clases o detecciones
    ###   - scores_personas: lista de scores de confianza para detecciones de personas
    ###   - todas_clases: lista de IDs de clases detectadas
    ###   - todos_scores: lista de scores de todas las detecciones
    ###   - max_score: score más alto entre todas las detecciones
    ###   - En modelos semánticos: porcentaje de píxeles de clase persona, total de clases únicas
    def detectar_en_imagen(self, imagen: Image.Image, umbrales: List[float]) -> Dict:
        ### Obtiene dimensiones de la imagen (orden correcto: width, height)
        w, h = imagen.size

        ### Preprocesa la imagen y la convierte en tensores para el modelo
        inputs = self.processor(images=imagen, return_tensors="pt").to(self.device)

        ### Ejecuta la inferencia sin cálculo de gradientes y mide el tiempo de ejecución
        inicio = time.time()
        with torch.no_grad():
            outputs = self.model(**inputs)
        tiempo_ms = (time.time() - inicio) * 1000

        ### Diccionario para almacenar resultados por umbral
        resultados = {'tiempo_inferencia_ms': tiempo_ms}

        ### Post-procesamiento para modelos semánticos
        if self.es_semantico:
            try:
                ### Obtiene la máscara semántica con clases por píxel
                ### target_sizes debe estar en formato (height, width)
                resultado_semantico = self.processor.post_process_semantic_segmentation(
                    outputs, target_sizes=[(h, w)]
                )[0]

                ### Extrae clases únicas presentes en la imagen
                unique_classes = torch.unique(resultado_semantico)

                ### Para cada umbral, se calcula si hay presencia significativa de la clase "persona"
                for umbral in umbrales:
                    ### Crea una máscara booleana donde los píxeles coinciden con la clase "persona"
                    persona_mask = (resultado_semantico == self.clase_persona)

                    ### Cuenta el número de píxeles de clase "persona"
                    persona_pixels = persona_mask.sum().item()
                    total_pixels = resultado_semantico.numel()

                    ### Calcula el porcentaje actual de píxeles de clase "persona"
                    porcentaje_actual = (persona_pixels / total_pixels) * 100
                    porcentaje_minimo = umbral * 100

                    ### Se considera que hay personas si se supera el umbral y hay más de 50 píxeles
                    personas = 1 if porcentaje_actual >= porcentaje_minimo and persona_pixels > 50 else 0

                    resultados[f'umbral_{umbral}'] = {
                        'personas': personas,
                        'total': len(unique_classes),
                        'todas_clases': unique_classes.tolist(),
                        'persona_pixels': persona_pixels,
                        'porcentaje_persona': porcentaje_actual,
                        'scores_personas': [1.0] if personas > 0 else []
                    }

            ### En caso de error, se registra en el resultado por cada umbral
            except Exception as e:
                for umbral in umbrales:
                    resultados[f'umbral_{umbral}'] = {
                        'error': str(e), 'personas': 0, 'total': 0
                    }

        ### Post-procesamiento para modelos de instancia
        else:
            for umbral in umbrales:
                try:
                    ### Aplica post-procesamiento con el umbral de score especificado
                    resultado = self.processor.post_process_instance_segmentation(
                        outputs, target_sizes=[(h, w)], threshold=umbral
                    )[0]

                    ### Extrae etiquetas y scores de las detecciones
                    labels = resultado.get("labels", torch.tensor([]))
                    scores = resultado.get("scores", torch.tensor([]))

                    ### Filtra detecciones que corresponden a la clase "persona"
                    personas = sum(1 for l in labels if int(l.item()) == self.clase_persona)

                    ### Extrae scores de las detecciones de personas
                    scores_personas = [
                        float(s.item()) for l, s in zip(labels, scores)
                        if int(l.item()) == self.clase_persona
                    ]

                    ### Extrae todas las clases y scores detectados
                    todas_clases = [int(l.item()) for l in labels]
                    todos_scores = [float(s.item()) for s in scores]

                    resultados[f'umbral_{umbral}'] = {
                        'personas': personas,
                        'total': len(labels),
                        'scores_personas': scores_personas,
                        'todas_clases': todas_clases,
                        'todos_scores': todos_scores,
                        'max_score': max(todos_scores) if todos_scores else 0.0
                    }

                ### En caso de error, se registra en el resultado por cada umbral
                except Exception as e:
                    resultados[f'umbral_{umbral}'] = {
                        'error': str(e), 'personas': 0, 'total': 0
                    }

        return

    ### Método para liberar memoria del modelo y procesador, útil en entornos con GPU limitada ###
    def limpiar(self):
        del self.model, self.processor
        torch.cuda.empty_cache()

DetectorPersonas definido


In [6]:
# ==========================================
# 4: PROCESADOR DE RESULTADOS
# ==========================================
class ProcesadorResultados:
    """Procesa y formatea resultados"""

    def __init__(self, output_path: str):
        self.output_path = Path(output_path)

    def procesar_imagen(self, ruta_imagen: str, detector: DetectorPersonas,
                       umbrales: List[float]) -> Dict:
        """Procesa una imagen completa"""
        inicio = time.time()

        try:
            # Preparar imagen
            imagen = Utils.preparar_imagen(ruta_imagen)
            hash_img = Utils.calcular_hash(ruta_imagen)

            # Información básica
            img_array = np.array(imagen)
            gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

            info_imagen = {
                'archivo': os.path.basename(ruta_imagen),
                'hash': hash_img,
                'tamaño': imagen.size,
                'brillo_promedio': float(np.mean(gray))
            }

            # Detectar
            resultados_deteccion = detector.detectar_en_imagen(imagen, umbrales)

            # Resultado final
            tiempo_total = (time.time() - inicio) * 1000

            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': info_imagen,
                'deteccion': resultados_deteccion,
                'tiempo_total_ms': tiempo_total,
                'modelo': detector.modelo_name,
                'exitoso': True
            }

        except Exception as e:
            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': {'archivo': os.path.basename(ruta_imagen)},
                'error': str(e),
                'exitoso': False
            }

    def mostrar_resumen(self, resultados: List[Dict]) -> None:
        """Muestra resumen de resultados"""
        exitosos = sum(1 for r in resultados if r.get('exitoso', False))
        print(f"\nRESUMEN: {exitosos}/{len(resultados)} imágenes procesadas")

        if exitosos == 0:
            return

        # Estadísticas por umbral
        for umbral in config.UMBRALES['agresivo']:
            personas_detectadas = []
            for r in resultados:
                if r.get('exitoso', False):
                    deteccion = r.get('deteccion', {})
                    umbral_data = deteccion.get(f'umbral_{umbral}', {})
                    personas_detectadas.append(umbral_data.get('personas', 0))

            if personas_detectadas:
                total_personas = sum(personas_detectadas)
                imagenes_con_personas = sum(1 for p in personas_detectadas if p > 0)
                print(f"  Umbral {umbral:5.3f}: {total_personas} personas en {imagenes_con_personas} imágenes")

print("ProcesadorResultados definido")

ProcesadorResultados definido


In [7]:
# ==========================================
# 5: FUNCIONES EJECUTABLES
# ==========================================

def ejecutar_evaluacion_basica(modelo_idx: int = 0, umbral_config: str = 'agresivo'):
    """
    FUNCIÓN EJECUTABLE: Evaluación básica

    Args:
        modelo_idx: Índice del modelo (0, 1, 2)
        umbral_config: Configuración de umbrales ('conservador', 'normal', 'agresivo', 'ultra')
    """
    print(f"EVALUACIÓN BÁSICA - Modelo {modelo_idx}, Umbrales: {umbral_config}")
    print("=" * 60)

    # Validar parámetros
    if modelo_idx >= len(config.MODELOS):
        print(f"Modelo index inválido. Máximo: {len(config.MODELOS)-1}")
        return

    if umbral_config not in config.UMBRALES:
        print(f"Configuración inválida. Disponibles: {list(config.UMBRALES.keys())}")
        return

    # Cargar datos
    imagenes = Utils.cargar_imagenes(config.DATASET_PATH)
    if not imagenes:
        print("No hay imágenes")
        return

    # Ejecutar
    modelo = config.MODELOS[modelo_idx]
    umbrales = config.UMBRALES[umbral_config]

    detector = DetectorPersonas(modelo)
    procesador = ProcesadorResultados(config.OUTPUT_PATH)

    # Procesar cada imagen
    resultados = []
    for i, ruta in enumerate(tqdm(imagenes, desc="Procesando")):
        print(f"[{i+1:3d}/{len(imagenes)}] {os.path.basename(ruta)}")
        resultado = procesador.procesar_imagen(ruta, detector, umbrales)
        resultados.append(resultado)

        # Log básico
        if resultado.get('exitoso', False):
            deteccion = resultado['deteccion']
            mejor_umbral = min(umbrales)  # Usar el más sensible
            datos = deteccion.get(f'umbral_{mejor_umbral}', {})
            personas = datos.get('personas', 0)
            total = datos.get('total', 0)
            print(f"{personas} personas de {total} detecciones")
        else:
            print(f"Error: {resultado.get('error', 'unknown')}")

    # Guardar y mostrar resumen
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archivo = config.OUTPUT_PATH / f"evaluacion_{modelo_idx}_{umbral_config}_{timestamp}.json"
    Utils.guardar_json(resultados, str(archivo))

    procesador.mostrar_resumen(resultados)
    detector.limpiar()

    return str(archivo)

def probar_imagen_individual(ruta_imagen: str, mostrar_detalles: bool = True):
    """
    FUNCIÓN EJECUTABLE: Probar una imagen específica

    Args:
        ruta_imagen: Ruta completa a la imagen
        mostrar_detalles: Si mostrar información detallada
    """
    print(f"PRUEBA INDIVIDUAL: {os.path.basename(ruta_imagen)}")
    print("=" * 60)

    if not os.path.exists(ruta_imagen):
        print(f"Imagen no encontrada: {ruta_imagen}")
        return

    # Usar modelo más potente y umbrales ultra sensibles
    detector = DetectorPersonas(config.MODELOS[0])  # El más grande
    procesador = ProcesadorResultados(config.OUTPUT_PATH)

    resultado = procesador.procesar_imagen(ruta_imagen, detector, config.UMBRALES['ultra'])

    if not resultado.get('exitoso', False):
        print(f"Error: {resultado.get('error')}")
        detector.limpiar()
        return

    # Mostrar resultados
    deteccion = resultado['deteccion']
    imagen_info = resultado['imagen']

    print(f"Imagen: {imagen_info['tamaño']} pixels, brillo: {imagen_info['brillo_promedio']:.1f}")
    print(f"Tiempo: {deteccion['tiempo_inferencia_ms']:.1f}ms")
    print()

    # Resultados por umbral
    print("DETECCIONES POR UMBRAL:")
    for umbral in config.UMBRALES['ultra']:
        datos = deteccion.get(f'umbral_{umbral}', {})
        if 'error' in datos:
            print(f"  {umbral:6.3f}: Error - {datos['error']}")
            continue

        personas = datos.get('personas', 0)
        total = datos.get('total', 0)
        max_score = datos.get('max_score', 0)

        print(f"  {umbral:6.3f}: {personas:2d} personas de {total:2d} total (max score: {max_score:.4f})")

        if mostrar_detalles and datos.get('todas_clases'):
            clases_unicas = sorted(set(datos['todas_clases']))
            print(f"             Clases: {clases_unicas}")

    # Recomendación
    mejor_umbral_data = deteccion.get('umbral_0.01', deteccion.get('umbral_0.001', {}))
    personas_detectadas = mejor_umbral_data.get('personas', 0)

    print(f"\nRESULTADO: {personas_detectadas} personas detectadas")
    if personas_detectadas == 0:
        total_detecciones = mejor_umbral_data.get('total', 0)
        if total_detecciones > 0:
            print(" Se detectaron otros objetos pero no personas")
            print("   Posibles soluciones:")
            print("   - Usar umbrales aún más bajos")
            print("   - Verificar que la imagen contenga personas visibles")
            print("   - Probar con otro modelo")
        else:
            print(" No se detectó ningún objeto")

    detector.limpiar()
    return resultado

def comparar_modelos_rapido(max_imagenes: int = 3):
    """
    FUNCIÓN EJECUTABLE: Comparación rápida de modelos

    Args:
        max_imagenes: Número máximo de imágenes a procesar
    """
    print(f"COMPARACIÓN RÁPIDA - {max_imagenes} imágenes máximo")
    print("=" * 60)

    imagenes = Utils.cargar_imagenes(config.DATASET_PATH)[:max_imagenes]
    if not imagenes:
        print("No hay imágenes")
        return

    umbrales = config.UMBRALES['agresivo']
    resultados_comparacion = {}

    for i, modelo in enumerate(config.MODELOS):
        print(f"\nModelo {i+1}/{len(config.MODELOS)}: {modelo.split('/')[-1]}")

        detector = DetectorPersonas(modelo)
        procesador = ProcesadorResultados(config.OUTPUT_PATH)

        resultados_modelo = []
        for ruta in imagenes:
            resultado = procesador.procesar_imagen(ruta, detector, umbrales)
            resultados_modelo.append(resultado)

        # Estadísticas rápidas
        exitosos = sum(1 for r in resultados_modelo if r.get('exitoso', False))

        personas_totales = 0
        tiempo_promedio = 0

        for resultado in resultados_modelo:
            if resultado.get('exitoso', False):
                # Usar umbral más sensible para contar
                deteccion = resultado['deteccion']
                umbral_data = deteccion.get('umbral_0.01', deteccion.get('umbral_0.05', {}))
                personas_totales += umbral_data.get('personas', 0)
                tiempo_promedio += resultado.get('tiempo_total_ms', 0)

        if exitosos > 0:
            tiempo_promedio /= exitosos

        resultados_comparacion[modelo] = {
            'exitosos': exitosos,
            'personas_totales': personas_totales,
            'tiempo_promedio_ms': tiempo_promedio
        }

        print(f" {exitosos}/{len(imagenes)} exitosas")
        print(f" {personas_totales} personas detectadas")
        print(f" {tiempo_promedio:.1f}ms promedio")

        detector.limpiar()

    # Resumen final
    print(f"\nRESUMEN COMPARACIÓN:")
    print("-" * 40)
    for modelo, datos in resultados_comparacion.items():
        nombre = modelo.split('/')[-1]
        print(f"{nombre:30s}: {datos['personas_totales']:2d} personas, {datos['tiempo_promedio_ms']:5.1f}ms")

    return resultados_comparacion

print("Funciones ejecutables definidas")

Funciones ejecutables definidas


In [11]:
## Ejecutamos evaluación básica.
for i in range(len(config.MODELOS)):
    for umbral in config.UMBRALES.keys():
        ejecutar_evaluacion_basica(i, umbral)

EVALUACIÓN BÁSICA - Modelo 0, Umbrales: ultra
Encontradas 1 imágenes
Cargando: facebook/mask2former-swin-large-coco-instance
Clase 0: person
Clase persona encontrada: 0 = person
Modelo cargado en cuda (semántico: False)


Procesando:   0%|          | 0/1 [00:00<?, ?it/s]

[  1/1] _DSC0147.jpg


Procesando: 100%|██████████| 1/1 [00:00<00:00,  1.80it/s]

0 personas de 0 detecciones
Guardado: /content/drive/MyDrive/TFM/mask2former/resultados/evaluacion_0_ultra_20250902_184211.json

RESUMEN: 1/1 imágenes procesadas
  Umbral 0.001: 0 personas en 0 imágenes
  Umbral 0.010: 0 personas en 0 imágenes
  Umbral 0.050: 0 personas en 0 imágenes
  Umbral 0.100: 0 personas en 0 imágenes
  Umbral 0.300: 0 personas en 0 imágenes
EVALUACIÓN BÁSICA - Modelo 0, Umbrales: agresivo
Encontradas 1 imágenes
Cargando: facebook/mask2former-swin-large-coco-instance


...

Clase 0: person
Clase persona encontrada: 0 = person
Modelo cargado en cuda (semántico: False)


Procesando:   0%|          | 0/1 [00:00<?, ?it/s]

[  1/1] _DSC0147.jpg


Procesando: 100%|██████████| 1/1 [00:00<00:00,  3.36it/s]

0 personas de 0 detecciones
Guardado: /content/drive/MyDrive/TFM/mask2former/resultados/evaluacion_2_conservador_20250902_184230.json

RESUMEN: 1/1 imágenes procesadas
  Umbral 0.001: 0 personas en 0 imágenes
  Umbral 0.010: 0 personas en 0 imágenes
  Umbral 0.050: 0 personas en 0 imágenes
  Umbral 0.100: 0 personas en 0 imágenes
  Umbral 0.300: 0 personas en 0 imágenes
